# 9.Find the date on which the entire population will get at least one does of vaccination. Assume the same rate of vaccination as in the week ending on 14th Aug, 2021.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta 
import os
import math

In [2]:
census_2011 ="Census2011data.xlsx"
df_census=pd.read_excel(census_2011,usecols= 'A,G,H,I,K')

In [3]:
Name_changes={"JAMMU & KASHMIR": "Jammu and Kashmir",
               "ANDAMAN & NICOBAR ISLANDS": "Andaman and Nicobar Islands",
               "DADRA & NAGAR HAVELI": "Dadra and Nagar Haveli and Daman and Diu",
               "DAMAN & DIU": "Dadra and Nagar Haveli and Daman and Diu","Mumbai Suburban":"Mumbai"}
state_id_change={26:25}

df_census.replace({'Name': Name_changes,"State":state_id_change},inplace= True)

In [4]:
df_census=df_census.groupby(["State","Level","Name","TRU"], as_index=False)["TOT_P"].sum()

In [5]:
df_census = df_census[df_census['Level'] == 'DISTRICT']
df_census = df_census[df_census['TRU'] == 'Total']
df_census = df_census[['Name','TRU','TOT_P']]

In [6]:
rename_dist = {'Allahabad ':'Prayagraj','Ahmadabad':'Ahmedabad', 'East District':'East Sikkim',
        'North  District': 'North Sikkim', 'South District':'South Sikkim','West District':'West Sikkim','Garhwal':'Pauri Garhwal',
         'Gulbarga':'Kalaburagi','Ahmadnagar':'Ahmednagar','Almora\n':'Almora','Amritsar ':'Amritsar',
       'Anand  ': 'Anand','Anugul  ':'Angul','Ariyalur  ':'Ariyalur','Badgam':'Budgam','Bagalkot ':'Bagalkote',
        'Baleshwar':'Balasore','Banas Kantha':'Banaskantha','Bandipore':'Bandipora','Bangalore Rural':'Bengaluru Rural',
        'Bangalore':'Bengaluru Urban','Bijapur':'Vijayapura',
        'Bankura ':'Bankura','Bara Banki':'Barabanki','Baramula':'Baramulla',"Barddhaman ":"Barddhaman",'Barwani ':'Barwani','Baudh':'Boudh',
        'Belgaum':'Belagavi','Bellary':'Ballari', 'Bid':'Beed','Bulandshahr ':'Bulandshahr','Buldana':'Buldhana',
        'Central':'Central Delhi','Chamarajanagar':'Chamarajanagara','Chikmagalur':'Chikkamagaluru',
         'Chittaurgarh':'Chittorgarh', 'Dadra & Nagar Haveli':'Dadra and Nagar Haveli', 'Darjiling ':'Darjeeling',
         'Debagarh':'Deogarh', 'Dhamtari ':'Dhamtari', 'Dhaulpur':'Dholpur', 'Dimapur ':'Dimapur', 'Dindori ':'Dindori',
        'Dohad  ':'Dahod', 'East':'East Delhi', 'Faizabad':'Ayodhya', 'Firozpur':'Ferozepur', 'Ganganagar ':'Ganganagar',
         'Garhwa ':'Garhwa', 'Gondiya':'Gondia', 'Gurgaon':'Gurugram', 'Haora ':'Howrah', 'Harda ':'Harda',
         'Hardwar':'Haridwar', 'Hugli ':'Hooghly', 'Idukki ':'Idukki','Jagatsinghapur ':'Jagatsinghpur',
         'Jaintia Hills':'West Jaintia Hills', 'Jajapur  ':'Jajpur', 'Jalaun ':'Jalaun', 'Jalor':'Jalore',
         'Jalpaiguri ':'Jalpaiguri', 'Janjgir - Champa':'Janjgir Champa', 'Jashpur ':'Jashpur', 'Jehanabad ':'Jehanabad',
        'Jhunjhunun':'Jhunjhunu', 'Jyotiba Phule Nagar':'Amroha', 'Kachchh':'Kutch', 'Kaimur (Bhabua)':'Kaimur',
         'Kanniyakumari':'Kanyakumari', 'Kanshiram Nagar':'Kasganj', 'Kapurthala ':'Kapurthala', 'Karur ':'Karur',
         'Katni ':'Katni', 'Kendrapara ': 'Kendrapara', 'Khandwa (East Nimar)':'Khandwa','Khargone (West Nimar)':'Khargone',
         'Kheri':'Lakhimpur Kheri', 'Khordha ':'Khordha', 'Koch Bihar ':'Cooch Behar', 'Kodarma':'Koderma', 'Korba ':'Korba',
         'Lahul & Spiti':'Lahaul and Spiti', 'Leh(Ladakh)':'Leh', 'Mahamaya Nagar':'Hathras', 'Mahbubnagar':'Mahabubnagar',
        'Mahesana':'Mehsana', 'Mahrajganj':'Maharajganj', 'Maldah ':'Malda', 'Malkangiri  ':'Malkangiri', 'Mewat ':'Nuh',
         'Muktsar':'Sri Muktsar Sahib', 'Murshidabad ':'Murshidabad', 'Mysore':'Mysuru', 'Nabarangapur ':'Nabarangapur',
         'Nadia ':'Nadia', 'Nagapattinam  ': 'Nagapattinam','Namakkal   ':'Namakkal', 'Narsimhapur':'Narsinghpur', 'Navsari  ':'Navsari', 'Nayagarh  ':'Nayagarh',
         'Neemuch ':'Neemuch', 'North':'North Delhi', 'North  & Middle Andaman':'North and Middle Andaman', 'North East':'North East Delhi',
         'North Twenty Four Parganas':'North 24 Parganas', 'North West':'North West Delhi', 'Palwal ':'Palwal',
         'Panch Mahals':'Panchmahal','Pashchim Champaran':'West Champaran', 'Pashchimi Singhbhum':'West Singhbhum', 'Patan  ':'Patan',
         'Perambalur  ':'Perambalur', 'Porbandar ':'Porbandar', 'Purba Champaran':'East Champaran', 'Purbi Singhbhum': 'East Singhbhum',
         'Puruliya':'Purulia', 'Rangareddy':'Ranga Reddy',"Raigarh":"Raigad",'Rayagada  ':'Rayagada','Sabar Kantha':'Sabarkantha', 'Sahibzada Ajit Singh Nagar':'S.A.S. Nagar', 'Sant Ravidas Nagar (Bhadohi)':'Bhadohi',
         'Shahid Bhagat Singh Nagar ':'Shahid Bhagat Singh Nagar', 'Sheopur ':'Sheopur', 'Shimoga':'Shivamogga',
         'Shupiyan': 'Shopiyan', 'South':'South Delhi','South Tripura ':'South Tripura', 'South Twenty Four Parganas':'South 24 Parganas', 'South West':'South West Delhi',
         'Sri Potti Sriramulu Nellore':'S.P.S. Nellore', 'Tamenglong ':'Tamenglong', 'The Dangs':'Dang',
         'The Nilgiris':'Nilgiris', 'Theni  ': 'Theni', 'Tirunelveli ':'Tirunelveli', 'Tumkur':'Tumakuru',
         'West':'West Delhi', 'West Tripura ':'West Tripura', 'Y.S.R.':'Y.S.R. Kadapa'}

In [7]:
df_census.replace({'Name':rename_dist},inplace=True)
df_census['Name'] = df_census['Name'].str.upper()

In [8]:
#Census data frame with required details
df_census

,Name,TRU,TOT_P
4,ANANTNAG,Total,1078692
7,BUDGAM,Total,753745
10,BANDIPORA,Total,392232
13,BARAMULLA,Total,1008039
16,DODA,Total,409936
...,...,...,...
2002,PUDUCHERRY,Total,950289
2005,YANAM,Total,55626
2011,NICOBARS,Total,36842
2014,NORTH AND MIDDLE ANDAMAN,Total,105597


In [9]:
cowin_data = 'cowin_vaccine_data_districtwise.csv'

In [10]:
vac_data = pd.read_csv(cowin_data,index_col=0,usecols=['S No','State_Code','State','District','07/08/2021.3','14/08/2021.3'],low_memory=False)

In [11]:
vac_data

,State_Code,State,District,07/08/2021.3,14/08/2021.3
S No,,,,,
NaN,NaN,NaN,NaN,First Dose Administered,First Dose Administered
1.0,AN,Andaman and Nicobar Islands,Nicobars,21918,22081
2.0,AN,Andaman and Nicobar Islands,North and Middle Andaman,63392,68043
3.0,AN,Andaman and Nicobar Islands,South Andaman,129711,142585
4.0,AP,Andhra Pradesh,Anantapur,1259410,1364374
...,...,...,...,...,...
751.0,WB,West Bengal,Purulia,528833,580596
752.0,WB,West Bengal,Birbhum,353927,381719
753.0,WB,West Bengal,South 24 Parganas,1185731,1289167


In [12]:
vac_data = vac_data[vac_data['14/08/2021.3'] != 'First Dose Administered']

In [13]:
vac_data = vac_data.rename(columns = {'14/08/2021.3':'First Doses till 14thAug21','07/08/2021.3' : 'First Doses till 7thAug21'})

In [14]:
vac_data

,State_Code,State,District,First Doses till 7thAug21,First Doses till 14thAug21
S No,,,,,
1.0,AN,Andaman and Nicobar Islands,Nicobars,21918,22081
2.0,AN,Andaman and Nicobar Islands,North and Middle Andaman,63392,68043
3.0,AN,Andaman and Nicobar Islands,South Andaman,129711,142585
4.0,AP,Andhra Pradesh,Anantapur,1259410,1364374
5.0,AP,Andhra Pradesh,Chittoor,1443079,1565591
...,...,...,...,...,...
751.0,WB,West Bengal,Purulia,528833,580596
752.0,WB,West Bengal,Birbhum,353927,381719
753.0,WB,West Bengal,South 24 Parganas,1185731,1289167


In [15]:
vac_data['State'] = vac_data['State'].str.upper()
vac_data['District'] = vac_data['District'].str.upper()

In [16]:
dist_changes={"WARANGAL RURAL":"WARANGAL","WARANGAL URBAN":"WARANGAL","KOMARAM BHEEM":"ADILABAD","MANCHERIAL":"ADILABAD",
"NIRMAL":"ADILABAD","WANAPARTHY":"MAHABUBNAGAR","NAGARKURNOOL":"MAHABUBNAGAR","KAMAREDDY":"NIZAMABAD","SANGAREDDY":"MEDAK",
"SIDDIPET":"MEDAK","BHADRADRI KOTHAGUDEM":"KHAMMAM","JAGTIAL":"KARIMNAGAR","JANGAON":"NALGONDA",
"JAYASHANKAR BHUPALAPALLY":"KARIMNAGAR","JOGULAMBA GADWAL":"MAHABUBNAGAR","MEDCHAL MALKAJGIRI":"RANGA REDDY",
"MULUGU":"WARANGAL","NARAYANPET":"MAHABUBNAGAR",
"PEDDAPALLI":"KARIMNAGAR","RAJANNA SIRCILLA":"KARIMNAGAR","SURYAPET":"NALGONDA","VIKARABAD":"RANGA REDDY",
              "YADADRI BHUVANAGIRI":"NALGONDA","MAHABUBABAD":"WARANGAL"
             }

In [17]:
vac_data.replace({'District': dist_changes},inplace= True)

In [18]:
vac_data.replace({'District': {"PASCHIM BARDHAMAN":"BARDDHAMAN","PURBA BARDHAMAN":"BARDDHAMAN"}},inplace= True)

In [19]:
vac_data=vac_data[['State_Code','District','First Doses till 7thAug21','First Doses till 14thAug21']]

In [20]:
vac_data["First Doses till 14thAug21"]=vac_data["First Doses till 14thAug21"].astype(int)
vac_data["First Doses till 7thAug21"]=vac_data["First Doses till 7thAug21"].astype(int)
vac_data=vac_data.groupby(["State_Code","District"], as_index=False).agg({'First Doses till 7thAug21':'sum','First Doses till 14thAug21':'sum'}).reset_index()

<ipython-input-1-ba318c7cf226>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vac_data["First Doses till 14thAug21"]=vac_data["First Doses till 14thAug21"].astype(int)
<ipython-input-1-ba318c7cf226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vac_data["First Doses till 7thAug21"]=vac_data["First Doses till 7thAug21"].astype(int)


In [21]:
vac_data

,index,State_Code,District,First Doses till 7thAug21,First Doses till 14thAug21
0,0,AN,NICOBARS,21918,22081
1,1,AN,NORTH AND MIDDLE ANDAMAN,63392,68043
2,2,AN,SOUTH ANDAMAN,129711,142585
3,3,AP,ANANTAPUR,1259410,1364374
4,4,AP,CHITTOOR,1443079,1565591
...,...,...,...,...,...
700,700,WB,PASCHIM MEDINIPUR,887384,948146
701,701,WB,PURBA MEDINIPUR,1414132,1539152
702,702,WB,PURULIA,528833,580596
703,703,WB,SOUTH 24 PARGANAS,1877838,2067595


In [22]:
data_final  = pd.merge(df_census,vac_data,how='inner',left_on=['Name'],right_on=['District'])

In [23]:
data_final=data_final[['District',"TOT_P","State_Code","First Doses till 7thAug21","First Doses till 14thAug21"]]

In [24]:
data_final.sort_values("TOT_P",axis=0,ascending = True,inplace = True ,kind = 'quicksort',na_position = 'last' )
data_final.drop_duplicates(subset ="TOT_P",keep='last', inplace=True)

In [25]:
data_final

,District,TOT_P,State_Code,First Doses till 7thAug21,First Doses till 14thAug21
255,ANJAW,21167,AR,10826,10922
34,LAHAUL AND SPITI,31564,HP,22854,26939
266,UPPER SIANG,35320,AR,17143,17475
644,NICOBARS,36842,AN,21918,22081
641,MAHE,41816,PY,30276,30907
...,...,...,...,...,...
529,PUNE,9429408,MH,4873003,5097676
563,BENGALURU URBAN,9621551,KA,7031810,7263343
349,NORTH 24 PARGANAS,10009781,WB,2937394,3197404
535,THANE,11060148,MH,2753209,2898733


In [26]:
data_final=data_final.groupby(["State_Code"], as_index=False).agg({'TOT_P':'sum','First Doses till 14thAug21':'sum','First Doses till 7thAug21':'sum'}).reset_index()

In [27]:
data_final

,index,State_Code,TOT_P,First Doses till 14thAug21,First Doses till 7thAug21
0,0,AN,380581,232709,215021
1,1,AP,49386799,18516141,17477123
2,2,AR,1375723,552299,542644
3,3,AS,31205576,10257764,9154326
4,4,BR,104099452,25167378,22945519
5,5,CH,1055450,733579,686609
6,6,CT,23795984,6365264,6183043
7,7,DL,16787941,6901107,6482028
8,8,DN,586956,598380,580287
9,9,GA,1458545,1112815,1087385


In [28]:
data_final['rateofvaccination']=(data_final['First Doses till 14thAug21']-data_final['First Doses till 7thAug21']).astype(float)/7

In [29]:
data_final['populationleft']= data_final['TOT_P']-data_final['First Doses till 14thAug21']
data_final['Due days to Atleast one dose']=(data_final['populationleft'])/data_final['rateofvaccination']
data_final['Due days to Atleast one dose']=data_final['Due days to Atleast one dose'].apply(np.ceil).astype(int)

In [30]:
data_final.rename(columns={'State_Code':"stateid"},inplace=True)
data_final=data_final[['stateid','populationleft','rateofvaccination','Due days to Atleast one dose']]

In [31]:
data_final['Present_date']='2021-08-14'
data_final['populationleft']=data_final['populationleft'].astype(int)
data_final['Due days to Atleast one dose']=data_final['Due days to Atleast one dose'].astype(int)
num = data_final._get_numeric_data()
num[num < 0] = 0

<ipython-input-1-4b0555782f61>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['Present_date']='2021-08-14'
<ipython-input-1-4b0555782f61>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['populationleft']=data_final['populationleft'].astype(int)
<ipython-input-1-4b0555782f61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [32]:
data_final['Present_date'] = pd.to_datetime(data_final['Present_date']) #Convert the first column to pandas datetime type
data_final['date'] = data_final['Present_date']+pd.to_timedelta(data_final['Due days to Atleast one dose'],unit='d') #Convert the second column type to pandas timedelta type and add to first column (which is already in date time type)
data_final

<ipython-input-1-f8844a62a39c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['Present_date'] = pd.to_datetime(data_final['Present_date']) #Convert the first column to pandas datetime type
<ipython-input-1-f8844a62a39c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final['date'] = data_final['Present_date']+pd.to_timedelta(data_final['Due days to Atleast one dose'],unit='d') #Convert the second column type to pandas timedelta type and add to first column (which is already in date time t

,stateid,populationleft,rateofvaccination,Due days to Atleast one dose,Present_date,date
0,AN,147872,2526.857143,59,2021-08-14,2021-10-12
1,AP,30870658,148431.142857,208,2021-08-14,2022-03-10
2,AR,823424,1379.285714,597,2021-08-14,2023-04-03
3,AS,20947812,157634.000000,133,2021-08-14,2021-12-25
4,BR,78932074,317408.428571,249,2021-08-14,2022-04-20
5,CH,321871,6710.000000,48,2021-08-14,2021-10-01
6,CT,17430720,26031.571429,670,2021-08-14,2023-06-15
7,DL,9886834,59868.428571,166,2021-08-14,2022-01-27
8,DN,0,2584.714286,0,2021-08-14,2021-08-14
9,GA,345730,3632.857143,96,2021-08-14,2021-11-18


In [33]:
data_final=data_final[['stateid','populationleft','rateofvaccination','date']]
data_final.sort_values("stateid",axis=0,ascending = True,inplace = True ,kind = 'quicksort',na_position = 'last' )

<ipython-input-1-b242631eeb58>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_final.sort_values("stateid",axis=0,ascending = True,inplace = True ,kind = 'quicksort',na_position = 'last' )


In [34]:
data_final.to_csv('./Output_Files/Q9/complete-vaccination.csv',index=False)